In [1]:
import codecs
import re
import numpy as np
from bs4 import BeautifulSoup
from sklearn.cross_validation import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from konlpy.tag import Mecab
mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")  # window에서 Mecab을 사용하기 위해 설정 후 사용합니다.

C:\Users\dukhyun\AppData\Local\conda\conda\envs\keras\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# 데이터 불러오기
with codecs.open('Citation_Data.txt', 'r', 'utf-8') as f:
    docs = f.read()
bs = BeautifulSoup(docs, "lxml")
docsList = bs.find_all('c')
del docs

In [3]:
# 데이터 정리
docs = []
for doc in docsList:
    cited = doc.get('citedarticle')
    citing = doc.get('citingarticle')
    clue = doc.get('clue')
    sentiment = doc.get('sentiment')
    sentiments = [str(l.text) for l in doc.find_all('s')] # l.text로 인해 target tag 역시 사라집니다.
    docs.append([cited, citing, sentiment, clue, ' '.join(sentiments)])

In [4]:
# Clue, Sentiments, Sentiment로 데이터 재정리
pnn = {'POS':2, 'NEG':1, 'NEU':0, 'NEU,NEU':0}
X = np.array([l[4] for l in docs])
Y = np.array([pnn[l[2]] for l in docs])
clue = np.array([l[3] for l in docs])

In [5]:
# 데이터를 4/5는 train set, 1/5는 test set으로 이용합니다.(비율을 맞춰 나눕니다.)
# 우선 반복을 5회 하지는 않습니다.
skf = StratifiedKFold(Y, n_folds=5, shuffle=True)

for train, test in skf:
    trainX, trainY, trainClue = X[train], Y[train], clue[train]
    testX, testY, testClue = X[test], Y[test], clue[test]
    break

In [6]:
# Train Clue를 단어 Feature로 사용하기 위하여 형태소 분석을하여 단어 사전을 구축합니다.
# 형태소 분석기는 Mecab을 이용합니다.
word_dict = []
if True:  # True로 하면 Clue만을 이용하여 word_dict를 구성합니다.
    for c in trainClue:
        if c:
            word_dict += [i[0]+'/'+i[1] for i in mecab.pos(c.replace(' ', ''))]

    word_dict = set(word_dict)
    word_dict = {w:i for i, w in enumerate(word_dict)}
elif True: # 위에서 False일때이며, 문장에서 단어를 뽑아내 feature로 사용합니다.
    word_dict = []
    for doc in trainX:
        word_dict += [i[0]+'/'+i[1] for i in mecab.pos(doc.replace(' ', '')) if i[1] != 'SY']
    word_dict = set(word_dict)
    word_dict = {w:i for i, w in enumerate(word_dict)}

In [7]:
# 데이터를 넣을 공간을 만듭니다.
train_X = np.zeros((len(trainY), len(word_dict)), dtype=np.int32)
test_X = np.zeros((len(testY), len(word_dict)), dtype=np.int32)

In [8]:
# 데이터들의 문장을 띄어쓰기를 제거하고 형태소 분석하고, 기타 기호 SY 형태소를 제거하였음
# 띄어쓰기를 제거한 이유는 다음과 같습니다.
# 문장을 읽어올때 잘못된 띄어쓰기 문제가 발생하였기 때문에 띄어쓰기를 제거하여 형태소 분석을 합니다. 
# Train Set
for i, doc in enumerate(trainX):
    doc = [i[0]+'/'+i[1] for i in mecab.pos(doc.replace(' ', '')) if i[1] != 'SY']
    for w in doc:
        try:
            train_X[word_dict[w], i] += 1
        except:
            pass

# Test Set
for i, doc in enumerate(testX):
    doc = [i[0]+'/'+i[1] for i in mecab.pos(doc.replace(' ', '')) if i[1] != 'SY']
    for w in doc:
        try:
            test_X[word_dict[w], i] += 1
        except:
            pass

In [9]:
# NaiveBayes
clf = GaussianNB()
clf.fit(train_X, trainY)
print("train accuracy: ", len(trainY[clf.predict(train_X) == trainY]) / len(trainY))
print("accuracy: ", len(testY[clf.predict(test_X) == testY]) / len(testY))
print("\n", classification_report(testY, clf.predict(test_X), target_names=['NEU', 'NEG', 'POS']))

train accuracy:  0.26371681415929205
accuracy:  0.3591549295774648

              precision    recall  f1-score   support

        NEU       0.78      0.31      0.44       101
        NEG       0.27      0.20      0.23        15
        POS       0.19      0.65      0.29        26

avg / total       0.61      0.36      0.39       142



In [10]:
# SVM Classification
clf = LinearSVC()
clf.fit(train_X, trainY)
print("train accuracy: ", len(trainY[clf.predict(train_X) == trainY]) / len(trainY))
print("accuracy: ", len(testY[clf.predict(test_X) == testY]) / len(testY))
print("\n", classification_report(testY, clf.predict(test_X), target_names=['NEU', 'NEG', 'POS']))

train accuracy:  0.8601769911504424
accuracy:  0.5774647887323944

              precision    recall  f1-score   support

        NEU       0.69      0.78      0.73       101
        NEG       0.13      0.13      0.13        15
        POS       0.08      0.04      0.05        26

avg / total       0.52      0.58      0.55       142



In [11]:
# LogisticRegression
clf = LogisticRegression()
clf.fit(train_X, trainY)
print("train accuracy: ", len(trainY[clf.predict(train_X) == trainY]) / len(trainY))
print("accuracy: ", len(testY[clf.predict(test_X) == testY]) / len(testY))
print("\n", classification_report(testY, clf.predict(test_X), target_names=['NEU', 'NEG', 'POS']))

train accuracy:  0.8265486725663717
accuracy:  0.676056338028169

              precision    recall  f1-score   support

        NEU       0.71      0.93      0.80       101
        NEG       0.50      0.07      0.12        15
        POS       0.14      0.04      0.06        26

avg / total       0.58      0.68      0.59       142

